In [ ]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
import difflib


## Loading CSVs

In [ ]:
ml_xlsx_file_sample = reviewers_xlsx_file_sample = None
ml_xlsx_file = reviewers_xlsx_file = None

In [72]:
### Original files used for notebook
REPO_PATH = '/home/celo/Desktop/git-repo-slr/SLR-Automated_selection_of_studies'
# ml_xlsx_file_path = 'output/svm-only/kfold-anovaf/zenodo-table.xlsx'

# RESULTS_PATH = '/home/celo/Desktop/data-results'
# reviewers_xlsx_file_path = 'open-repo-papers-zenodo/full-table.xlsx'


### New results
# ml_xlsx_file_path = 'output-v6/gridsearch_pearson_fs/scoring_recall_macro/k1100-report-oct15-14h28m.xlsx'
ml_xlsx_file_path = 'output-v6/gridsearch_anovaf_fs/scoring_f1/k1200-report-oct15-15h09m.xlsx'
RESULTS_PATH = '/home/celo/Desktop/'
reviewers_xlsx_file_path = 'final-changes-reviewers-set/results-final-versions/imutable-zenodo-columns.xlsx'

# reviewers_xlsx_file = '/home/mcostalonga/new-home/thesis/open-repo-papers-zenodo/full-table.xlsx' # Attempt1 | Sheet1

# ml_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/ml_test.xlsx'
# reviewers_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/reviewers_test.xlsx'

# ml_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/ml_test_big_sample.xlsx'
# reviewers_xlsx_file_sample = '/home/mcostalonga/new-home/thesis/git-repo-slr/SLR-Automated_selection_of_studies/output/svm-only/kfold-anovaf/reviewers_test_big_sample.xlsx'

ml_xlsx_file = os.path.join(REPO_PATH, ml_xlsx_file_path)
reviewers_xlsx_file = os.path.join(RESULTS_PATH, reviewers_xlsx_file_path)


In [73]:
# Sample files to test notebook
if ml_xlsx_file_sample and reviewers_xlsx_file_sample:
    ml_pd = pd.read_excel(ml_xlsx_file_sample, sheet_name='Probabilities', engine='openpyxl')
    desired_columns = ["Titles", "Claes", "Marcos", "Katia"]
    reviewers_pd = pd.read_excel(reviewers_xlsx_file_sample, sheet_name='Probabilities', engine='openpyxl', usecols=desired_columns)

# Real complete files
elif ml_xlsx_file and reviewers_xlsx_file:
    ml_pd = pd.read_excel(ml_xlsx_file, sheet_name='Probabilities', engine='openpyxl')
    desired_columns = ["Titles", "Claes", "Marcos", "Katia"]
    # reviewers_pd = pd.read_excel(reviewers_xlsx_file, sheet_name='Sheet1', engine='openpyxl', usecols=desired_columns)
    reviewers_pd = pd.read_excel(reviewers_xlsx_file, sheet_name='notebook-set', engine='openpyxl', usecols=desired_columns)

else:
    print('[ERROR] Inform valid CSVs files to read')
    raise Exception

ml_pd = ml_pd.iloc[: ,1:]  # Only execute this once to remove the first unused column


In [74]:
# OBS: Titles with typos written on Testing sets 
# # Apply this just to make sure they don't go unoticed

# ml_titles_with_typos = {
#     'A COLABORA{\c{C}}{\~A}O UNIVERSIDADE-EMPRESA NO CONTEXTO DA IND{\'U}STRIA 4.0': 'A COLABORAÇÃO UNIVERSIDADE-EMPRESA NO CONTEXTO DA INDÚSTRIA 4.0'
# }
ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\c{C}}', 'Ç')
ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\~A}', 'Ã')
ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\'U}', 'Ú')


/tmp/ipykernel_23502/3175091779.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\c{C}}', 'Ç')
/tmp/ipykernel_23502/3175091779.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\\~A}', 'Ã')
/tmp/ipykernel_23502/3175091779.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_pd['Titles'] = ml_pd['Titles'].str.replace(r'{\'U}', 'Ú')


In [75]:
# ML's CSV summary
print('[ML] Num of rows:', len(ml_pd))
ml_pd.head(15)


[ML] Num of rows: 570


,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba
0,Operations Technology and Organizational Struc...,0,1,0.882744,0.453457
1,Design science as nested problem solving,0,0,0.280341,0.280659
2,A Large-Scale Empirical Study of Practitioners...,0,0,0.113759,0.197997
3,Towards an Approach Matching CMD and DSR to Im...,0,1,0.974311,0.551916
4,A PROPOSAL FOR USING DESIGN SCIENCE IN EDUCATI...,0,0,0.428776,0.373863
5,University-industry collaboration and open sou...,0,1,0.541670,0.489848
6,A bibliometric analysis of the Turkish softwar...,0,1,0.782667,0.507440
7,An Improved Indoor Positioning Method Based on...,0,0,0.520822,0.098209
8,Industry-academia collaboration in software te...,0,1,1.000000,0.749822
9,Measuring Software Reliability: A Trend Using ...,0,0,0.073916,0.234321


In [76]:
new_reviewers_pd = deepcopy(reviewers_pd)

# TODO Nov 13: Finish this...
# string_to_numeric = {
#     'Assessed, citing SLR': 0,
#     'Assessed original SLR': 0,
#     'Assessed Batch 1': 0,
#     'Assessed': 0,
#     # 'PhD thesis': 0,
#     # 'Bachelor thesis': 0,
#     # 'Licentiate thesis': 0,
#     # 'Not English': 0,
#     'Original SLR': 0,   # Only one wrongly result
#     # 'Book': 0,
#     # 'Book (chapter in own book)': 0,
#     # 'SMS': 0,
#     # 'SLR': 0,
#     # 'Keynote': 0,
#     'See line 55': 0,
#     'See line 63': 0,
#     'See line 66': 0,
#     'See line 108': 0,
#     'See line 178': 0,
#     'See line 205': 0,
#     # 'Literature review': 0,
#     # 'Review article': 0,
#     'Panel summary': 0,
#     'Included Assessed original SLR': 2,
#     'Included Assessed, citing SLR': 2,
#     'Included Assessed': 2
# }

string_to_numeric = {
    'Assessed, citing SLR': 0,
    'Assessed original SLR': 0,
    'Assessed Batch 1': 0,
    'Assessed': 0,
    'PhD thesis': 0,
    'Bachelor thesis': 0,
    'Licentiate thesis': 0,
    'Not English': 0,
    'Original SLR': 0,
    'Book': 0,
    'Book (chapter in own book)': 0,
    'SMS': 0,
    'SLR': 0,
    'Keynote': 0,
    'See line 55': 0,
    'See line 63': 0,
    'See line 66': 0,
    'See line 108': 0,
    'See line 178': 0,
    'See line 205': 0,
    'Literature review': 0,
    'Review article': 0,
    'Panel summary': 0,
    'Included Assessed original SLR': 2,
    'Included Assessed, citing SLR': 2,
    'Included Assessed': 2
}
for column in ['Claes', 'Marcos', 'Katia']:
    new_reviewers_pd[column] = new_reviewers_pd[column].replace(string_to_numeric)
# OBS: Line 7 should be all 0's after this change

# Reviewers' CSV summary
print('[Reviewers] Num of rows:', len(new_reviewers_pd))
new_reviewers_pd.head(10)

[Reviewers] Num of rows: 567


,Titles,Claes,Marcos,Katia
0,A research framework for building SPI proposal...,2,1,0
1,A Researcher's Experiences in Supporting Indus...,2,2,0
2,Characterizing industry-academia collaboration...,2,2,2
3,Context checklist for industrial software engi...,2,0,2
4,Continuous and collaborative technology transf...,2,2,2
5,Exploring the Dimensions of University-Company...,2,2,1
6,Guiding the selection of research methodology ...,0,2,2
7,Industry-academia collaboration for realism in...,2,2,2
8,Industry-Academia research collaboration in so...,2,2,2
9,Industry–academia collaborations in software t...,2,2,1


In [77]:
# Reviewers' CSV summary
print('[Reviewers] Num of rows:', len(reviewers_pd))
reviewers_pd.head(10)

[Reviewers] Num of rows: 567


,Titles,Claes,Marcos,Katia
0,A research framework for building SPI proposal...,2,1,0
1,A Researcher's Experiences in Supporting Indus...,2,2,0
2,Characterizing industry-academia collaboration...,2,2,2
3,Context checklist for industrial software engi...,2,0,2
4,Continuous and collaborative technology transf...,2,2,2
5,Exploring the Dimensions of University-Company...,2,2,1
6,Guiding the selection of research methodology ...,0,2,2
7,Industry-academia collaboration for realism in...,2,2,2
8,Industry-Academia research collaboration in so...,2,2,2
9,Industry–academia collaborations in software t...,2,2,1


In [78]:
# sorted_df = reviewers_pd.sort_values(by='Titles', ascending=True)
# sorted_df.head()
reviewers_pd = new_reviewers_pd
reviewers_pd.head(10)

,Titles,Claes,Marcos,Katia
0,A research framework for building SPI proposal...,2,1,0
1,A Researcher's Experiences in Supporting Indus...,2,2,0
2,Characterizing industry-academia collaboration...,2,2,2
3,Context checklist for industrial software engi...,2,0,2
4,Continuous and collaborative technology transf...,2,2,2
5,Exploring the Dimensions of University-Company...,2,2,1
6,Guiding the selection of research methodology ...,0,2,2
7,Industry-academia collaboration for realism in...,2,2,2
8,Industry-Academia research collaboration in so...,2,2,2
9,Industry–academia collaborations in software t...,2,2,1


#


#



# RETHINK OF THIS SOLUTION START

### Search for each title from ML in the Reviewers df to find closest match by DiffLib
##### Find how many matches can be found by this method

In [ ]:
# # matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
# i = 0
# # similarity_treshold = 85  
# # similarity_treshold = 75  
# similarity_treshold = 50   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
# difflib_treshold = similarity_treshold/100

# # Format both dataframes to lower case so is easier to match
# formated_ml_df = deepcopy(ml_pd)
# formated_ml_df['Titles_Formated'] = formated_ml_df['Titles'].str.lower()
# formated_ml_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN

# formated_reviewers_df = deepcopy(reviewers_pd)
# formated_reviewers_df['Titles_Formated'] = formated_reviewers_df['Titles'].str.lower()
# formated_reviewers_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN

# difflib_match_count = 0
# matches = dict()

# while(i < len(formated_ml_df)-1):
# # while(i < 3):
#     curr_ml_title = formated_ml_df.loc[i]['Titles_Formated']

#     # Match by difflib
#     difflib_match = difflib.get_close_matches(curr_ml_title, formated_reviewers_df['Titles_Formated'], n=1, cutoff=difflib_treshold)
   
#     if difflib_match:
#             # print('DIFF LIB:\n\t ml_title: {}\n\tmatch: {}'.format(curr_ml_title, difflib_match))
#             difflib_match_count+=1

#             # # Use boolean indexing to filter rows where 'Titles' column is equal to 'string_value'
#             matched_row_index = formated_reviewers_df[formated_reviewers_df['Titles_Formated'] == difflib_match[0]].index[0]

#             # Update the column '_match_index' on both dataframes to use to merge them later
#             formated_reviewers_df.at[matched_row_index, '_match_index'] =  difflib_match_count
#             formated_ml_df.at[i, '_match_index'] =  difflib_match_count


#             if difflib_match_count not in matches:
#                   matches[difflib_match_count] = { 'ml_title': curr_ml_title, 'reviewer_match': difflib_match[0] }

#             # print(formated_ml_df.loc[i].head())
#             # formated_ml_df.loc[i].head()
#             # print(formated_reviewers_df.loc[matched_row_index])
#             # formated_reviewers_df.loc[matched_row_index].head()
#             # print(matched_row_reviewers['Titles'], matched_row_reviewers['Claes'], matched_row_reviewers['Marcos'], matched_row_reviewers['Katia'])


#             # TODO: Change the original DataFrame so ML already have a value for each reviewer for matches found
#             # TODO: Need to add three new columns for each reviewer and complete with NaN initially...
#             # TODO: Then, will apply The Fuzz only to the rows that weren't matched...
            

#             # # Copy values from curr_row into next_row for the second and third columns
#             # similar_matches_df.iloc[i+1, 4:7] = similar_matches_df.iloc[i, 4:7]

#             # # Copy values from next_row into curr_row for the Reviewers columns
#             # similar_matches_df.iloc[i, 2:4] = similar_matches_df.iloc[i+1, 2:4]

#             # # Copy values from next_row into curr_row for the Reviewers columns
#             # similar_matches_df.iloc[i, 4:7] = similar_matches_df.iloc[i+1, 4:7]

#             # # Copy values from curr_row into next_row for the second and third columns
#             # similar_matches_df.iloc[i+1, 2:4] = similar_matches_df.iloc[i, 2:4]


#             # matches[curr_ml_title] = difflib_match[0]
#     i+=1

# print('Difflib matches:', difflib_match_count)
# # pd.DataFrame.from_dict(matches).head()


In [ ]:
# # Merge ML's and Reviewers' CSVs into one, by matching Titles

# # formated_reviewers_pd


# # tmp_df = pd.merge(formated_ml_df, formated_reviewers_df, on='_match_index', how='inner')
# tmp_df = pd.merge(formated_ml_df, formated_reviewers_df.drop(['Titles', 'Titles_Formated'], axis=1), on='_match_index', how='inner')

# # tmp_df['_match_index'].fillna(placeholder, inplace=True)
# # tmp_df = tmp_df.dropna(subset=['_match_index'])
# tmp_df = tmp_df.sort_values(by='_match_index', ascending=False)



# print(len(tmp_df))
# tmp_df.head(10)
# # Total of 632, but removing NaN is 537... HOW ??????
# # Maybe titles from REV that were not found/matched on ML... ??

In [ ]:
# total = 0
# # nan_rows = tmp_df.isna().any(axis=1)
# rows_with_nan = tmp_df[tmp_df.isna().any(axis=1)]

# # for i in nan_rows:
# #     if i:
# #         print(total, i)
# #         total+=1
# print(len(rows_with_nan))
# # print(rows_with_nan.loc[0])
# rows_with_nan.head()


In [ ]:
# DEBUG TESTING
# To remove columns...
# tmp_df2.

# Problema: Remover titles que estão no CSV dos reviewers mas não no do ML... 
# OU simplesmente deletar na mão

In [ ]:
# merged_dataframe = ml_pd.merge(reviewers_pd, on='Titles', how='outer')
# print(len(merged_dataframe))
# merged_dataframe.head(8)

###### Pandas Functions to Combine CSVs (merge, join...)

In [ ]:
# # Merge ML's and Reviewers' CSVs into one, by matching Titles

# # formated_reviewers_pd


# # tmp_df = pd.merge(formated_ml_df, formated_reviewers_df, on='_match_index', how='inner')
# tmp_df = pd.merge(formated_ml_df, formated_reviewers_df.drop(['Titles', 'Titles_Formated'], axis=1), on='_match_index', how='inner')

# # tmp_df['_match_index'].fillna(placeholder, inplace=True)
# tmp_df = tmp_df.dropna(subset=['_match_index'])


# print(len(tmp_df))
# tmp_df.head(10)

In [ ]:
# # Merge ML's and Reviewers' CSVs into one, by matching Titles
# merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles', how='outer')
# print(len(merged_dataframe))
# merged_dataframe.head(8)

# RETHINK OF THIS SOLUTION END

#



#

### Pandas Functions to Combine CSVs (merge, join...)

In [ ]:
# # OBS: NEW CELL Sep27

# # TODO: Ideia ao invés de fazer merge pelo "Titles" e depois percorrer dois ponteiros comparando o str.lower()
# #       Já fazer o merge no ["titles"].lower() e depois percorrer dataset comparando "ML_titles" com "Reviewers_titles"

# # Convert both titles to lower case and then merge it
# ml_pd['Titles_Formated'] = ml_pd['Titles'].str.lower()
# reviewers_pd['Titles_Formated'] = reviewers_pd['Titles'].str.lower()

# merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles_Formated', how='outer')

# # Rename columns Titles_x and Titles_y
# merged_dataframe['Titles_ML'] = merged_dataframe['Titles_x'] 
# merged_dataframe['Titles_Reviewers'] = merged_dataframe['Titles_y'] 
# merged_dataframe = merged_dataframe.drop(['Titles_x', 'Titles_y'], axis=1)

# merged_dataframe_sorted = merged_dataframe.sort_values(by='Titles_Formated', ascending=True)
# merged_dataframe_sorted.head(20)

# print(len(merged_dataframe_sorted))
# merged_dataframe_sorted.head(8)


# # merged_dataframe_sorted = merged_dataframe_sorted[["Was Selected?	SVM_proba	Titles_Formated	Claes	Marcos	Katia	Titles_ML	Titles_Reviewers"]]
# # # Sorts the csv by title so that missmatch titles are next to each other
# # merged_dataframe_sorted = merged_dataframe.sort_values(by='Titles_Formated', ascending=True)


In [79]:
# Merge ML's and Reviewers' CSVs into one, by matching Titles
merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles', how='outer')
print(len(merged_dataframe))
merged_dataframe.head(8)

674


,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia
0,Operations Technology and Organizational Struc...,0.0,1.0,0.882744,0.453457,NaN,NaN,NaN
1,Design science as nested problem solving,0.0,0.0,0.280341,0.280659,NaN,NaN,NaN
2,A Large-Scale Empirical Study of Practitioners...,0.0,0.0,0.113759,0.197997,0.0,0.0,0.0
3,Towards an Approach Matching CMD and DSR to Im...,0.0,1.0,0.974311,0.551916,NaN,NaN,NaN
4,A PROPOSAL FOR USING DESIGN SCIENCE IN EDUCATI...,0.0,0.0,0.428776,0.373863,0.0,0.0,0.0
5,University-industry collaboration and open sou...,0.0,1.0,0.541670,0.489848,0.0,0.0,2.0
6,A bibliometric analysis of the Turkish softwar...,0.0,1.0,0.782667,0.507440,0.0,0.0,0.0
7,An Improved Indoor Positioning Method Based on...,0.0,0.0,0.520822,0.098209,NaN,NaN,NaN


In [80]:
# Add new column with formated titles to sort by it
merged_dataframe['Titles_Formated'] = merged_dataframe['Titles'].str.lower()

# Sorts the csv by title so that missmatch titles are next to each other
merged_dataframe_sorted = merged_dataframe.sort_values(by='Titles_Formated', ascending=True)
merged_dataframe_sorted.head(20)

,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia,Titles_Formated
150,'Let's not reinvent the wheel': A qualitative ...,0.0,1.0,0.957646,0.317925,NaN,NaN,NaN,'let's not reinvent the wheel': a qualitative ...
6,A bibliometric analysis of the Turkish softwar...,0.0,1.0,0.782667,0.507440,0.0,0.0,0.0,a bibliometric analysis of the turkish softwar...
490,A case study of industry--academia communicati...,1.0,1.0,0.973800,0.630595,NaN,NaN,NaN,a case study of industry--academia communicati...
570,A case study of industry–academia communicatio...,NaN,NaN,NaN,NaN,2.0,0.0,2.0,a case study of industry–academia communicatio...
48,A case study on artefact-based RE improvement ...,0.0,0.0,0.176554,0.219512,0.0,0.0,0.0,a case study on artefact-based re improvement ...
309,A Chatbot for goal-oriented requirements modeling,0.0,0.0,0.262046,0.250289,0.0,0.0,0.0,a chatbot for goal-oriented requirements modeling
521,A collaborative autoethnographic analysis of i...,0.0,1.0,0.999999,0.653878,0.0,1.0,2.0,a collaborative autoethnographic analysis of i...
317,A Collaborative Method for Identification and ...,0.0,0.0,0.037940,0.144284,0.0,0.0,0.0,a collaborative method for identification and ...
52,A comparative study of software process improv...,0.0,0.0,0.969508,0.246491,0.0,0.0,0.0,a comparative study of software process improv...
393,A comprehensive software project management fr...,0.0,0.0,0.168292,0.225311,0.0,0.0,0.0,a comprehensive software project management fr...


#### Separate the rows containing NaN values from the Titles that were equal in both dataframes

In [81]:
nan_rows = merged_dataframe_sorted.isna().any(axis=1)
missmatched_df = merged_dataframe[nan_rows]
missmatched_df = missmatched_df.sort_values(by='Titles_Formated', ascending=True)
missmatched_df.head()

/tmp/ipykernel_23502/3866861985.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missmatched_df = merged_dataframe[nan_rows]


,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia,Titles_Formated
150,'Let's not reinvent the wheel': A qualitative ...,0.0,1.0,0.957646,0.317925,NaN,NaN,NaN,'let's not reinvent the wheel': a qualitative ...
490,A case study of industry--academia communicati...,1.0,1.0,0.973800,0.630595,NaN,NaN,NaN,a case study of industry--academia communicati...
570,A case study of industry–academia communicatio...,NaN,NaN,NaN,NaN,2.0,0.0,2.0,a case study of industry–academia communicatio...
92,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.0,0.550480,0.406230,NaN,NaN,NaN,a critical view on plm/alm convergence in prac...
583,A critical view on PLM/ALM convergence in prac...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...


In [82]:
# Resets the index so you can loop throught it using from i=0 to N like an array
missmatched_df = missmatched_df.reset_index()
# missmatched_df = missmatched_df.drop(columns='level_0')
missmatched_df.head(20)


,index,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia,Titles_Formated
0,150,'Let's not reinvent the wheel': A qualitative ...,0.0,1.0,0.957646,0.317925,NaN,NaN,NaN,'let's not reinvent the wheel': a qualitative ...
1,490,A case study of industry--academia communicati...,1.0,1.0,0.973800,0.630595,NaN,NaN,NaN,a case study of industry--academia communicati...
2,570,A case study of industry–academia communicatio...,NaN,NaN,NaN,NaN,2.0,0.0,2.0,a case study of industry–academia communicatio...
3,92,A Critical View on PLM/ALM Convergence in Prac...,0.0,0.0,0.550480,0.406230,NaN,NaN,NaN,a critical view on plm/alm convergence in prac...
4,583,A critical view on PLM/ALM convergence in prac...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...
5,419,A framework to improve university--industry co...,1.0,0.0,0.960124,0.498493,NaN,NaN,NaN,a framework to improve university--industry co...
6,571,A framework to improve university–industry col...,NaN,NaN,NaN,NaN,0.0,2.0,2.0,a framework to improve university–industry col...
7,584,A method to assess and argue for practical sig...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,a method to assess and argue for practical sig...
8,473,A Method to Assess and Argue for Practical Sig...,0.0,0.0,0.990007,0.369247,NaN,NaN,NaN,a method to assess and argue for practical sig...
9,117,A Practical Guide to Select Quality Indicators...,0.0,0.0,0.500000,0.331671,NaN,NaN,NaN,a practical guide to select quality indicators...


### Compare strings

In [83]:
# pip install thefuzz
from thefuzz import fuzz, process
from copy import deepcopy

In [84]:
def merge_not_NaN_values(df: pd.DataFrame, index):
    # print(df.loc[index])
    if np.isnan(df.loc[index, 'SVM_proba']) and not(np.isnan(df.loc[index, 'Marcos'])):
        # print('Row 1 is Reviewers and Row 2 is ML')
        return True
    elif np.isnan(df.loc[index, 'Marcos']) and not(np.isnan(df.loc[index, 'SVM_proba'])):
        # print('Row 2 is Reviewers and Row 1 is ML')
        return False
    else:
        raise KeyError('ERROR on merge_not_NaN_values')

In [ ]:
similar_matches_df.iloc[0,6:9]

In [85]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  # 176 missing papers...
# similarity_treshold = 75  # 176 missing papers...
similarity_treshold = 50  # Reduced to 110 missing papers...

similar_matches_df = deepcopy(missmatched_df)
similar_matches_df['Similarity'] = np.nan  # Add new column and set all rows to be NaN

fuzz_match_count = 0
while(i < len(similar_matches_df)-1):
    # print(i)
    curr_row = similar_matches_df.loc[i]['Titles_Formated']
    next_row = similar_matches_df.loc[i+1]['Titles_Formated']

    # Attempt to find match by using the fuzz
    similarity = fuzz.partial_ratio(curr_row, next_row) 

    if similarity >= similarity_treshold: # Same title
        fuzz_match_count+=1
        if merge_not_NaN_values(similar_matches_df, i): # Row i is Reviewers and Row i+1 is ML
            # Copy values from curr_row into next_row for the second and third columns
            # similar_matches_df.iloc[i+1, 4:7] = similar_matches_df.iloc[i, 4:7]
            similar_matches_df.iloc[i+1, 6:9] = similar_matches_df.iloc[i, 6:9]

            # Copy values from next_row into curr_row for the Reviewers columns
            similar_matches_df.iloc[i, 2:6] = similar_matches_df.iloc[i+1, 2:6]

        else: # Row i is ML and Row i+1 is Reviewers
            # Copy values from next_row into curr_row for the Reviewers columns
            similar_matches_df.iloc[i, 6:9] = similar_matches_df.iloc[i+1, 6:9]

            # Copy values from curr_row into next_row for the second and third columns
            similar_matches_df.iloc[i+1, 2:6] = similar_matches_df.iloc[i, 2:6]


        # Copy index and title from curr_row to next_row
        similar_matches_df.iloc[i, 0:2] = similar_matches_df.iloc[i+1, 0:2]
        similar_matches_df.at[i, 'Similarity'] = similar_matches_df.at[i+1, 'Similarity'] = similarity
        i+=2

    else: # Not the same
        i+=1

print('Fuzz matches:', fuzz_match_count)
similar_matches_df.head()


Fuzz matches: 102


,index,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,150,'Let's not reinvent the wheel': A qualitative ...,0.0,1.0,0.957646,0.317925,NaN,NaN,NaN,'let's not reinvent the wheel': a qualitative ...,NaN
1,570,A case study of industry–academia communicatio...,1.0,1.0,0.973800,0.630595,2.0,0.0,2.0,a case study of industry--academia communicati...,98.0
2,570,A case study of industry–academia communicatio...,1.0,1.0,0.973800,0.630595,2.0,0.0,2.0,a case study of industry–academia communicatio...,98.0
3,583,A critical view on PLM/ALM convergence in prac...,0.0,0.0,0.550480,0.406230,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0
4,583,A critical view on PLM/ALM convergence in prac...,0.0,0.0,0.550480,0.406230,0.0,0.0,0.0,a critical view on plm/alm convergence in prac...,100.0


In [ ]:
# np.isnan(new_df.iloc[19, 2])
# new_df.iloc[1, 4:7]
# new_df.head()

In [ ]:
similar_matches_df.head(20)

#### Combine new matches with other dataframe

In [86]:
# Get the first matches
original_matched_df = merged_dataframe_sorted.dropna(how='any')
# print(len(original_matched_df))
# original_matched_df.head()


# Get only the new rows that got a match
new_matches_df = similar_matches_df[(~similar_matches_df['Was Selected?'].isna()) & (~similar_matches_df['SVM_proba'].isna()) 
                                         & (~similar_matches_df['Claes'].isna()) & (~similar_matches_df['Marcos'].isna()) 
                                         & (~similar_matches_df['Katia'].isna())] 
# Drop duplicated rows from new matches (get df with unique Titles)
columns_to_check = ['index', 'Titles']
new_matches_df = new_matches_df[~new_matches_df.duplicated(subset=columns_to_check, keep='first')]
# print(len(new_matches_df))
# new_matches_df.head()              


In [ ]:
print(len(original_matched_df))
original_matched_df.head()

In [ ]:
print(len(new_matches_df))
new_matches_df.head()

#### Create new DF combining all matches (this should be equal to ALL titles used in the testing set for ML)

In [87]:
# Merge dataframe with all results
# columns_to_merge = ['Titles', 'Was Selected?', 'SVM_proba', 'Claes', 'Marcos', 'Katia']
columns_to_merge = ['Titles', 'Was Selected?', 'DT_proba', 'SVM_proba', 'RFOREST_proba', 'Claes', 'Marcos', 'Katia']
merged_df = original_matched_df.merge(new_matches_df[columns_to_merge], on=columns_to_merge, how='outer')


In [88]:
# Remove the last column 'Titles_Formated' from final dataframe
merged_df = merged_df.drop(columns='Titles_Formated')
print(len(merged_df))
merged_df.head()

564


,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia
0,A bibliometric analysis of the Turkish softwar...,0.0,1.0,0.782667,0.507440,0.0,0.0,0.0
1,A case study on artefact-based RE improvement ...,0.0,0.0,0.176554,0.219512,0.0,0.0,0.0
2,A Chatbot for goal-oriented requirements modeling,0.0,0.0,0.262046,0.250289,0.0,0.0,0.0
3,A collaborative autoethnographic analysis of i...,0.0,1.0,0.999999,0.653878,0.0,1.0,2.0
4,A Collaborative Method for Identification and ...,0.0,0.0,0.037940,0.144284,0.0,0.0,0.0


#### Verify DF with unmatched ML titles (this shouldn't happen, check for specific titles to know what went wrong)


In [89]:
# Filtered ML (results that still may be missing...)
not_found_ml_titles_df = similar_matches_df[(~similar_matches_df['Was Selected?'].isna()) & (~similar_matches_df['SVM_proba'].isna()) 
                                         & (similar_matches_df['Claes'].isna()) & (similar_matches_df['Marcos'].isna())]  
print(len(not_found_ml_titles_df))
not_found_ml_titles_df.head(20)

6


,index,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia,Titles_Formated,Similarity
0,150,'Let's not reinvent the wheel': A qualitative ...,0.0,1.0,0.957646,0.317925,NaN,NaN,NaN,'let's not reinvent the wheel': a qualitative ...,NaN
63,1,Design science as nested problem solving,0.0,0.0,0.280341,0.280659,NaN,NaN,NaN,design science as nested problem solving,NaN
139,0,Operations Technology and Organizational Struc...,0.0,1.0,0.882744,0.453457,NaN,NaN,NaN,operations technology and organizational struc...,NaN
169,464,Self-Adaptation Driven by SysML and Goal Model...,0.0,1.0,0.742700,0.219140,NaN,NaN,NaN,self-adaptation driven by sysml and goal model...,NaN
202,164,Using a Visual Abstract as a Lens for Communic...,0.0,0.0,0.104434,0.283636,NaN,NaN,NaN,useful business cases: value creation in is pr...,50.0
203,164,Using a Visual Abstract as a Lens for Communic...,0.0,0.0,0.104434,0.283636,NaN,NaN,NaN,using a visual abstract as a lens for communic...,50.0


### Use difflib to search for each ML unmatched title the closest match on the REV df

In [90]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  
# similarity_treshold = 75  
# similarity_treshold = 50   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
similarity_treshold = 80   # 567 matches (only missed 21... REMEMBER THAT ML CSV IS BIGGER THAN REVIEWERS'...)
difflib_treshold = similarity_treshold/100


# Format both dataframes to lower case so is easier to match
formated_ml_df = deepcopy(not_found_ml_titles_df)
formated_ml_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN
formated_ml_df = formated_ml_df.drop(['Similarity'], axis=1)
formated_ml_df = formated_ml_df.reset_index()

formated_reviewers_df = deepcopy(reviewers_pd)
formated_reviewers_df['Titles_Formated'] = formated_reviewers_df['Titles'].str.lower()
formated_reviewers_df['_match_index'] = np.nan  # Add new column and set all rows to be NaN

difflib_match_count = 0
matches = dict()

while(i < len(formated_ml_df)-1):
# while(i < 3):
    curr_ml_title = formated_ml_df.loc[i]['Titles_Formated']

    # Match by difflib
    difflib_match = difflib.get_close_matches(curr_ml_title, formated_reviewers_df['Titles_Formated'], n=1, cutoff=difflib_treshold)
   
    if difflib_match:
            # print('DIFF LIB:\n\t ml_title: {}\n\tmatch: {}'.format(curr_ml_title, difflib_match))
            difflib_match_count+=1

            # # Use boolean indexing to filter rows where 'Titles' column is equal to 'string_value'
            matched_row_index = formated_reviewers_df[formated_reviewers_df['Titles_Formated'] == difflib_match[0]].index[0]

            # Update the column '_match_index' on both dataframes to use to merge them later
            formated_reviewers_df.at[matched_row_index, '_match_index'] =  difflib_match_count
            formated_ml_df.at[i, '_match_index'] =  difflib_match_count


            if difflib_match_count not in matches:
                  matches[difflib_match_count] = { 'ml_title': curr_ml_title, 'reviewer_match': difflib_match[0] }
    i+=1

print('Difflib matches:', difflib_match_count)
# pd.DataFrame.from_dict(matches).head()


Difflib matches: 1


In [ ]:
for c,index in enumerate(matches):
    if c < 7:
        print('ORIGINAL:   ', matches[index]['ml_title'])
        print('REMATCHE:   ', matches[index]['reviewer_match'])
        print('\n')


In [ ]:
for c,index in enumerate(matches):
    if c >= 7 and c < 14:
        print('ORIGINAL:   ', matches[index]['ml_title'])
        print('REMATCHE:   ', matches[index]['reviewer_match'])
        print('\n')


In [ ]:
for c,index in enumerate(matches):
    if c >= 14:
        print('ORIGINAL:   ', matches[index]['ml_title'])
        print('REMATCHE:   ', matches[index]['reviewer_match'])
        print('\n')

#### Get the sub dataframe of each one to merge the matches

In [91]:
# Reviewers dataframe
new_re_matches = formated_reviewers_df
new_re_matches = new_re_matches.dropna(subset=['_match_index'])
print('New matches count:', len(new_re_matches))
new_re_matches = new_re_matches.reset_index()
new_re_matches.head()

New matches count: 1


,index,Titles,Claes,Marcos,Katia,Titles_Formated,_match_index
0,38,Let's not reinvent the wheel': A qualitative i...,0,0,0,let's not reinvent the wheel': a qualitative i...,1.0


In [92]:
# ML dataframe
print('Old missmatch count:', len(formated_ml_df))
new_ml_matches = formated_ml_df
new_ml_matches = new_ml_matches.dropna(subset=['_match_index'])
print('New matches count:', len(new_ml_matches))
new_ml_matches = new_ml_matches.iloc[: ,2:]
new_ml_matches = new_ml_matches.reset_index()
new_ml_matches.head()

Old missmatch count: 6
New matches count: 1


,index,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia,Titles_Formated,_match_index
0,0,'Let's not reinvent the wheel': A qualitative ...,0.0,1.0,0.957646,0.317925,NaN,NaN,NaN,'let's not reinvent the wheel': a qualitative ...,1.0


##### Merge the new dataframes to get the new matches

In [93]:
# First join the results
new_matches_df = pd.merge(new_ml_matches.drop(['Claes', 'Marcos', 'Katia', 'index'], axis=1), 
                          new_re_matches.drop(['Titles', 'Titles_Formated', 'index'], axis=1), on='_match_index', how='outer')

print(len(new_matches_df))
new_matches_df.head()

1


,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Titles_Formated,_match_index,Claes,Marcos,Katia
0,'Let's not reinvent the wheel': A qualitative ...,0.0,1.0,0.957646,0.317925,'let's not reinvent the wheel': a qualitative ...,1.0,0,0,0


##### Append the new matches to the rest of them

In [94]:
# Now append the new matches to the rest
# columns_to_merge = ['Titles', 'Was Selected?', 'SVM_proba', 'Claes', 'Marcos', 'Katia']
columns_to_merge = ['Titles', 'Was Selected?', 'DT_proba', 'SVM_proba', 'RFOREST_proba', 'Claes', 'Marcos', 'Katia']
merged_df = merged_df.merge(new_matches_df[columns_to_merge], on=columns_to_merge, how='outer')

print(len(merged_df))
merged_df.head()



565


,Titles,Was Selected?,DT_proba,SVM_proba,RFOREST_proba,Claes,Marcos,Katia
0,A bibliometric analysis of the Turkish softwar...,0.0,1.0,0.782667,0.507440,0.0,0.0,0.0
1,A case study on artefact-based RE improvement ...,0.0,0.0,0.176554,0.219512,0.0,0.0,0.0
2,A Chatbot for goal-oriented requirements modeling,0.0,0.0,0.262046,0.250289,0.0,0.0,0.0
3,A collaborative autoethnographic analysis of i...,0.0,1.0,0.999999,0.653878,0.0,1.0,2.0
4,A Collaborative Method for Identification and ...,0.0,0.0,0.037940,0.144284,0.0,0.0,0.0


## Save file 

In [95]:
RESULTS_PATH = '/home/celo/Desktop/final-changes-reviewers-set'
# csv_file_path = '/home/mcostalonga/new-home/thesis/open-repo-papers-zenodo/zenodo-final-result/output3.csv'
csv_file_path = os.path.join(RESULTS_PATH, 'zenodo-tables-v6-f1-32-rf.csv')
excel_file_path = os.path.join(RESULTS_PATH, 'zenodo-tables-v6-f1-32-rf.xlsx')

# Convert the DataFrame to a CSV file
merged_df.to_csv(csv_file_path, index=False)  # Set index=False to omit the index column
merged_df.to_excel(excel_file_path, index=False)  # Set index=False to omit the index column

print(f'DataFrame has been successfully saved to {csv_file_path}')

DataFrame has been successfully saved to /home/celo/Desktop/final-changes-reviewers-set/zenodo-tables-v6-f1-32-rf.csv


#### Verify AGAIN DF with unmatched ML titles (this shouldn't happen, check for specific titles to know what went wrong)


In [ ]:
# ML dataframe

# Filtered ML (results that still may be missing...)
not_found_ml_titles_df = formated_ml_df[(formated_ml_df['_match_index'].isna())]  
print(len(formated_ml_df))
print(len(not_found_ml_titles_df))
not_found_ml_titles_df.head(20)

# print('Old missmatch count:', len(formated_ml_df))
# new_ml_matches = formated_ml_df
# new_ml_matches = new_ml_matches.dropna(subset=['_match_index'])
# print('New matches count:', len(new_ml_matches))
# new_ml_matches = new_ml_matches.iloc[: ,2:]
# new_ml_matches = new_ml_matches.reset_index()
# new_ml_matches.head()

In [ ]:
# TODO: NEW CODE Oct 1 Analysis of missing papers

missing_papers_path = '/home/celo/Desktop/final-changes-reviewers-set'
# csv_file_path = '/home/mcostalonga/new-home/thesis/open-repo-papers-zenodo/zenodo-final-result/output3.csv'
csv_file_path = os.path.join(missing_papers_path, '38_missing_papers.csv')

# Convert the DataFrame to a CSV file
not_found_ml_titles_df.to_csv(csv_file_path, index=False)  # Set index=False to omit the index column


In [ ]:
formated_ml_df.head()

In [ ]:
# # tmp_df = pd.merge(formated_ml_df, formated_reviewers_df, on='_match_index', how='inner')
# tmp_df = pd.merge(formated_ml_df, formated_reviewers_df.drop(['Titles', 'Titles_Formated'], axis=1), on='_match_index', how='inner')

# # tmp_df['_match_index'].fillna(placeholder, inplace=True)
# # tmp_df = tmp_df.dropna(subset=['_match_index'])
# tmp_df = tmp_df.sort_values(by='_match_index', ascending=False)


# # Merge ML's and Reviewers' CSVs into one, by matching Titles
# merged_dataframe = pd.merge(ml_pd, reviewers_pd, on='Titles', how='outer')
# print(len(merged_dataframe))
# merged_dataframe.head(8)

#### Verify DF with unmatched Reviewers' titles (this may happen, reviewers had more titles than our dataset)


In [ ]:
# Filtered Reviewers (results that still may be missing...)
not_found_reviewers_titles_df = similar_matches_df[(similar_matches_df['Was Selected?'].isna()) & (similar_matches_df['SVM_proba'].isna()) 
                                         & (~similar_matches_df['Claes'].isna()) & (~similar_matches_df['Marcos'].isna())]  
print(len(not_found_reviewers_titles_df))
not_found_reviewers_titles_df.head(20)




In [ ]:
# After dropping duplicateds...
unique_not_found_reviewers_titles_df = \
    not_found_reviewers_titles_df[~not_found_reviewers_titles_df.duplicated(subset=columns_to_check, keep='first')]

print(len(unique_not_found_reviewers_titles_df))
unique_not_found_reviewers_titles_df.head(20)

Fazer comparação de strings row a row:
- Comparar i com i++
- Usar thefuzz

--------------------------------------------------------------------------------------------------------------------

Caso de excesso no ML_pd (papers que precisam ser encontrados no Reviewers_pd mas não foram)
- Pensar em fazer nova busca para encontrá-los
- Was Selected? | SVM_proba != NaN
- Reviewers == NaN

Caso de excesso no Reviewers_pd (papers que foram avaliados pelos revisores mas não foram usados no testing_set)
- Descartar
- Was Selected? | SVM_proba == NaN
- Reviewers != NaN

----------------------------------------------------------------------------------------------------------------------

- Fazer novo merge no pandas que tinha dado match nos títulos (matched_df), pode ser concatenação 
- Reordenar pelos títulos 

Ao final, conferir se todos os X papers do ML.csv estão nesse novo pandas (verificar len)

### TODO: Criar caso de CSV mais complexo e testar para ver se script está funcionando...
- Possíveis problemas: Ordenação (ao ordenar estamos assumindo que os missmatchs estarão sempre um do lado do outro, o que pode não ser verdade caso Um paper tenha um caractere diferente no começo, esteja em lower x upper case...)

### Talvez a solução seja percorrer TODAS as linhas do CSV de REVIEWERS para CADA entrada do CSV de ML....

### Uma solução para o Upper x Lower case seria normalizar tudo para upper ou lower (OBS: Aparentemente o sorted coloca A B C antes de a b c...)

### Testar com os dois CSVs de verdades...

### OBS: By analysing the others notebooks, we selected which xlsx files got the best results for each test case and then manually copied the path of the files here

In [ ]:
# matched_df = merged_dataframe.dropna(subset=merged_dataframe.columns[merged_dataframe.isna().any()].tolist())
i = 0
# similarity_treshold = 85  # 176 missing papers...
# similarity_treshold = 75  # 176 missing papers...
similarity_treshold = 50  # Reduced to 110 missing papers...

difflib_treshold = similarity_treshold/100

similar_matches_df = deepcopy(missmatched_df)
similar_matches_df['Similarity'] = np.nan  # Add new column and set all rows to be NaN

difflib_match_count = 0
fuzz_match_count = 0

while(i < len(similar_matches_df)-1):
    # print(i)
    curr_row = similar_matches_df.loc[i]['Titles']
    next_row = similar_matches_df.loc[i+1]['Titles']

    # Attempt to find match by using the fuzz
    similarity = fuzz.partial_ratio(curr_row, next_row) 
    if similarity > similarity_treshold:
        fuzz_match_count+=1

    # If the fuzz did not found a match try by difflib
    difflib_match = difflib.get_close_matches(curr_row, similar_matches_df['Titles'], n=1, cutoff=difflib_treshold)
    if difflib_match:
            # print('DIFF LIB:', difflib_match, similarity)
            difflib_match_count+=1

    # TODO: Can think of a loop starting from higher treshold to lower and attempting multiple matches...

    if similarity >= similarity_treshold: # TODO: or difflib_match: # Same title
        # if merge_not_NaN_values(similar_matches_df, i):
        #     # Copy values from curr_row into next_row for the second and third columns
        #     similar_matches_df.iloc[i+1, 4:7] = similar_matches_df.iloc[i, 4:7]

        #     # Copy values from next_row into curr_row for the Reviewers columns
        #     similar_matches_df.iloc[i, 2:4] = similar_matches_df.iloc[i+1, 2:4]

        # else:
        #     # Copy values from next_row into curr_row for the Reviewers columns
        #     similar_matches_df.iloc[i, 4:7] = similar_matches_df.iloc[i+1, 4:7]

        #     # Copy values from curr_row into next_row for the second and third columns
        #     similar_matches_df.iloc[i+1, 2:4] = similar_matches_df.iloc[i, 2:4]


        # # Copy index and title from curr_row to next_row
        # similar_matches_df.iloc[i, 0:2] = similar_matches_df.iloc[i+1, 0:2]
        # similar_matches_df.at[i, 'Similarity'] = similar_matches_df.at[i+1, 'Similarity'] = similarity
        i+=2

    else: # Not the same
        i+=1

print('Fuzz matches:', fuzz_match_count)
print('Difflib matches:', difflib_match_count)

similar_matches_df.head()
